This is a stab at creating a churn prediction classifier.  I'm using a sample data set of 50K records from a ride sharing service. Predicting users that have a high churn risk enables the business to bucket these users separately for special promotional incentives, allocation of client service resources, etc to drive reengagement and prevent churn.         

In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [136]:
data = pd.read_csv('data/churn.csv')

In [137]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
avg_dist                  50000 non-null float64
avg_rating_by_driver      49799 non-null float64
avg_rating_of_driver      41878 non-null float64
avg_surge                 50000 non-null float64
city                      50000 non-null object
last_trip_date            50000 non-null object
phone                     49604 non-null object
signup_date               50000 non-null object
surge_pct                 50000 non-null float64
trips_in_first_30_days    50000 non-null int64
luxury_car_user           50000 non-null bool
weekday_pct               50000 non-null float64
dtypes: bool(1), float64(6), int64(1), object(4)
memory usage: 4.2+ MB


In [138]:
data.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4


In [139]:
#converting the date columns to datetime format
data.last_trip_date = pd.to_datetime(data.last_trip_date, format = '%Y-%m-%d')
data.signup_date = pd.to_datetime(data.signup_date, format = '%Y-%m-%d')
data.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4


In [140]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
avg_dist                  50000 non-null float64
avg_rating_by_driver      49799 non-null float64
avg_rating_of_driver      41878 non-null float64
avg_surge                 50000 non-null float64
city                      50000 non-null object
last_trip_date            50000 non-null datetime64[ns]
phone                     49604 non-null object
signup_date               50000 non-null datetime64[ns]
surge_pct                 50000 non-null float64
trips_in_first_30_days    50000 non-null int64
luxury_car_user           50000 non-null bool
weekday_pct               50000 non-null float64
dtypes: bool(1), datetime64[ns](2), float64(6), int64(1), object(2)
memory usage: 4.2+ MB


In [141]:
#defining churn as any user who hasn't used service in past month
data['churn'] = data.last_trip_date < '2014-06-01'

In [142]:
data.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,churn
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2,False
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0,True
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0,True
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0,False
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4,True


In [143]:
#~2/3 of users have churned
data.churn.value_counts()

True     31196
False    18804
Name: churn, dtype: int64

In [144]:
#replacing bool churn value with 1/0
data.churn = data['churn'].astype(int, inplace=True)

In [145]:
data.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,churn
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2,0
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0,1
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0,1
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0,0
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4,1


In [146]:
data.phone.value_counts()

iPhone     34582
Android    15022
Name: phone, dtype: int64

In [147]:
#a handful of records don't include a phone value.  Addding a placeholder here.
data.phone = data.phone.fillna('empty_phone')

In [148]:
#creating dummies for phone and city categorical vars
data.city = data.city.astype("category")
data.phone = data.phone.astype('category')

In [149]:
pd.crosstab(data.phone, data.city)

city,Astapor,King's Landing,Winterfell
phone,,,
Android,5244,2498,7280
empty_phone,121,64,211
iPhone,11169,7568,15845


In [150]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
avg_dist                  50000 non-null float64
avg_rating_by_driver      49799 non-null float64
avg_rating_of_driver      41878 non-null float64
avg_surge                 50000 non-null float64
city                      50000 non-null category
last_trip_date            50000 non-null datetime64[ns]
phone                     50000 non-null category
signup_date               50000 non-null datetime64[ns]
surge_pct                 50000 non-null float64
trips_in_first_30_days    50000 non-null int64
luxury_car_user           50000 non-null bool
weekday_pct               50000 non-null float64
churn                     50000 non-null int64
dtypes: bool(1), category(2), datetime64[ns](2), float64(6), int64(2)
memory usage: 4.0 MB


What to do about missing values?
-the ratings vars need to be removed, imputed with some value, or binned separately as having 'empty' value
-the first two options should be considered when the values are missing at random, otherwise we should bin
-to determine whether the missing values are random we can observe whether the missing values have an effect on churn by comparing the churn rate of users who have missing values v those who do not.  
*Disclaimer* I stole this technique from a very smart person in my Galvanize DSI cohort 

Some info on the scs function I'll use to compare the groups of records....

    scipy.stats.ttest_ind(a, b, axis=0, equal_var=True)[source]
    Calculates the T-test for the means of TWO INDEPENDENT samples of scores.

    This is a two-sided test for the null hypothesis that 2 independent samples have identical average (expected) 
    values. This test assumes that the populations have identical variances.

In [151]:
#writing a function to run t-test between groups that do and don't posess a particular condition
#used to determine whether the condition affects churn rate.  If not, we can either drop records with missing vals 
#or impute a value.  Otherwise, we should retain the records and bin them separately

import scipy.stats as scs

def t_test(feature, condition):
    '''
    INPUT feature (a pandas Series): the column of interest
    INPUT condition (boolean): condition to t-test by
    OUTPUT
    '''
    #test takes in a Series and applies the condition: the churn column where phone is null
    #compares this sample to the opposite: churn column where phone IS NOT null
    #compares the means of each sample to one another (mean of array of 1s/0s)
    #runs t-test 

    ttest = scs.ttest_ind(feature[condition], feature[-condition])
    print '***t-test for difference in means***'
    print 'user count with condition: {}, user count without condition: {}'\
        .format(len(feature[condition]), len(feature[-condition]))
    print 'mean with condition: {}, mean without condition: {}'\
        .format(feature[condition].mean(), feature[-condition].mean())
    print 't statistic: {}'.format(ttest.statistic, 4)
    print 'p-value: {}'.format(ttest.pvalue)
    
    

In [152]:
# Measure the effect of empty phone values
t_test(data['churn'], data['phone'] == 'empty_phone')

***t-test for difference in means***
user count with condition: 396, user count without condition: 49604
mean with condition: 0.664141414141, mean without condition: 0.623598903314
t statistic: 1.65893845728
p-value: 0.0971344664588


this condition does not generate a statistically significant difference in churn rate.  we can therefore drop the records with missing phone

In [153]:
data = data[data['phone'] != 'empty_phone']

In [154]:
#verifying dropped records
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49604 entries, 0 to 49999
Data columns (total 13 columns):
avg_dist                  49604 non-null float64
avg_rating_by_driver      49406 non-null float64
avg_rating_of_driver      41578 non-null float64
avg_surge                 49604 non-null float64
city                      49604 non-null category
last_trip_date            49604 non-null datetime64[ns]
phone                     49604 non-null category
signup_date               49604 non-null datetime64[ns]
surge_pct                 49604 non-null float64
trips_in_first_30_days    49604 non-null int64
luxury_car_user           49604 non-null bool
weekday_pct               49604 non-null float64
churn                     49604 non-null int64
dtypes: bool(1), category(2), datetime64[ns](2), float64(6), int64(2)
memory usage: 4.3 MB


In [155]:
#Measure the effect of missing avg_rating_by_driver values
t_test(data['churn'], data['avg_rating_by_driver'].isnull())

***t-test for difference in means***
user count with condition: 198, user count without condition: 49406
mean with condition: 0.818181818182, mean without condition: 0.622819090799
t statistic: 5.6644709906
p-value: 1.48293067402e-08


In [156]:
#Measure the effect of missing avg_rating_of_driver values
t_test(data['churn'], data['avg_rating_of_driver'].isnull())

***t-test for difference in means***
user count with condition: 8026, user count without condition: 41578
mean with condition: 0.797906802891, mean without condition: 0.589951416615
t statistic: 35.6534139171
p-value: 6.34574724075e-275


these conditions generate statistically significant differences in churn rate.  we therefore need to retain these records and information by bucketing the records separately

In [157]:
data.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,churn
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2,0
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0,1
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0,1
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0,0
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4,1


In [158]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49604 entries, 0 to 49999
Data columns (total 13 columns):
avg_dist                  49604 non-null float64
avg_rating_by_driver      49406 non-null float64
avg_rating_of_driver      41578 non-null float64
avg_surge                 49604 non-null float64
city                      49604 non-null category
last_trip_date            49604 non-null datetime64[ns]
phone                     49604 non-null category
signup_date               49604 non-null datetime64[ns]
surge_pct                 49604 non-null float64
trips_in_first_30_days    49604 non-null int64
luxury_car_user           49604 non-null bool
weekday_pct               49604 non-null float64
churn                     49604 non-null int64
dtypes: bool(1), category(2), datetime64[ns](2), float64(6), int64(2)
memory usage: 4.3 MB


In [159]:
def add_binned_ratings(df, old_col, new_col):
    '''
    Add column for binned ratings.
    
    INPUT:
    - df (full dataframe)
    - old_col (str): column name of average ratings
    - new_col (str): new column name for binned average ratings
    OUTPUT:
    - new dataframe
    '''
    df[new_col] = pd.cut(df[old_col].copy(), bins=[0., 3.99, 4.99, 5],
                            include_lowest=True, right=True)
    df[new_col].cat.add_categories('Missing', inplace=True)
    df[new_col].fillna('Missing', inplace=True)                        
    return df

data = add_binned_ratings(data, 'avg_rating_by_driver', 'bin_avg_rating_by_driver')
data = add_binned_ratings(data, 'avg_rating_of_driver', 'bin_avg_rating_of_driver')

In [160]:
data.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,churn,bin_avg_rating_by_driver,bin_avg_rating_of_driver
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2,0,"(4.99, 5]","(3.99, 4.99]"
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0,1,"(4.99, 5]","(4.99, 5]"
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0,1,"(4.99, 5]","(3.99, 4.99]"
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0,0,"(3.99, 4.99]","(3.99, 4.99]"
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4,1,"(3.99, 4.99]","(3.99, 4.99]"


In [161]:
data.bin_avg_rating_by_driver.value_counts()

(4.99, 5]       28271
(3.99, 4.99]    19573
[0, 3.99]        1562
Missing           198
Name: bin_avg_rating_by_driver, dtype: int64

In [162]:
data.bin_avg_rating_of_driver.value_counts()

(4.99, 5]       20625
(3.99, 4.99]    17165
Missing          8026
[0, 3.99]        3788
Name: bin_avg_rating_of_driver, dtype: int64

In [164]:
data.drop(['avg_rating_by_driver','avg_rating_of_driver'], axis=1, inplace=True)

In [165]:
data.head()

,avg_dist,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,churn,bin_avg_rating_by_driver,bin_avg_rating_of_driver
0,3.67,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2,0,"(4.99, 5]","(3.99, 4.99]"
1,8.26,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0,1,"(4.99, 5]","(4.99, 5]"
2,0.77,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0,1,"(4.99, 5]","(3.99, 4.99]"
3,2.36,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0,0,"(3.99, 4.99]","(3.99, 4.99]"
4,3.13,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4,1,"(3.99, 4.99]","(3.99, 4.99]"


EDA (working on this)

hypotheses
-price matters: avg_surge is correlated with churn, surge_pct correlated with churn
-high ratings matter: higher bin negatively correlates with churn 
-high usage matters: avg_dist * trips_in_first_30_days as proxy for total distance ridden, negatively correlated with churn
-phone matters: iPhone users have better experiences with app than Android users bc of phone software fragmentation
-high weekday usage matters: high weekday_pct might be an indicator that one is reliant on a ride share service for their commute, therefore less likely to churn
-initial usage habits matter: trips_in_first_30_days is negatievly correlated with churn
-lack of ratings by user could indicate underutilization of service features, low level utility
-lack of ratings by driver could indicate perception of poor customer experience

Pipeline (working on this)

Model Training and Selection (working on this)

Hyperparameter Tuning (coming soon)

TEST